# Download a public dataset

<big><b>File completed</b></big> (05/12/2021)  
Table of Contents + tool versions on 06/22/2021

Bénédicte Noblet

- Preparing session on IFB core cluster
- Online researches for a dataset
- Download data files with SRA Toolkit (*sra-tools*)
- Files and folders summary when leaving


---

## <b>Preparing session on IFB core cluster</b>

<em>loaded JupyterLab</em> : Version 2.2.9

In [1]:
echo "=== Cell launched on $(date) ==="

echo "=== Current IFB session size: Medium (4CPU, 10GB) or Large (10CPU, 50GB) ==="
jobid=$(squeue -hu $USER | awk '/jupyter/ {print $1}')
sacct --format=JobID,AllocCPUS,NODELIST -j ${jobid}

echo "=== Working's root folder is ==="
gohome="/shared/projects/gonseq/Building/"
echo "${gohome}"
echo ""

echo "=== current folder tree ==="
tree "${gohome}"
echo "=== current working directory ==="
echo "${PWD}"

=== Cell launched on Tue May 11 17:18:52 CEST 2021 ===
=== Current IFB session size: Medium (4CPU, 10GB) or Large (10CPU, 50GB) ===
       JobID  AllocCPUS        NodeList 
------------ ---------- --------------- 
16367476             10     cpu-node-47 
16367476.ba+         10     cpu-node-47 
16367476.0           10     cpu-node-47 
=== Working's root folder is ===
/shared/projects/gonseq/Building/

=== current folder tree ===
/shared/projects/gonseq/Building/
├── Data
│   └── info
│       ├── 16samples_SraRunTable.txt
│       ├── 16samples_SRR_Acc_List.txt
│       ├── SraRunTable.txt
│       └── SRR_Acc_List.txt
├── Pipeline
│   └── Pipe_1-bash_download-files.ipynb
└── Results

4 directories, 5 files
=== current working directory ===
/shared/ifbstor1/projects/gonseq/Building/Pipeline


---

If your datatest is already available on the server, skip this part... Unless you want to know how to do for next time ;)

## <b>I- Online researches</b>

Several cases can happen:
- you read a wonderful paper and want to analyze the dataset they use <br>
- you have a project but miss the data (and sequencing platform !) to do it <br>
- a enthusiastic PI ask you a question you don't yet have the answer <br>

For the two last options, you need to find a dataset before going on.

### **2- Find a paper dataset** <a id="section005"></a>

To that purpose, we can browse <a href="https://www.ncbi.nlm.nih.gov/gds"> <b>GEO datasets</b> </a> with some filters. <br>

<div class="alert alert-block alert-info">
    There is an alternative to the amercian NCBI database in Europe: <a href="https://www.ebi.ac.uk/ena/browser/home">ENA (<i>European Nucleotide Archive</i>)</a> from <a href="https://www.ebi.ac.uk/">EMBL-EBI</a> institute and part of the <a href="https://elixir-europe.org/platforms/data/core-data-resources">Elixir Core Data resource</a>.<br>
    Unfortunately, to the authors' knowledge, there is not any command-line tool to retrieve datafiles when a dataset is identified. So, we will use NCBI's resources for now.
</div>

Starting on the main page, proceed to a research with some (2 to 3, including RNAseq) key words about the research project you have. <br>
Once you face the huge amount of results, add following filters (left side bar of webpage) :
- *Organism* > *Customize* > *Browse* (Homo sapiens, Mus musculus, Rattus Norvegicus, ...) > select those you want to display > *Show*. Then click on the one or those you want
- *Study type* > Select only *Expression Profiling by high throughput sequencing*. Back on the results page, click on the added option to set it active
- If you still have too many options to handle, select *Tissue* in the *Attribute name* option

Then browse the results you have to find a paper that suits your project. <br>
When you pick up one, keep record of some access references:
<blockquote>
GEO Series Number: GSExxxxxx <br>
SRA related Number: SRPxxxxxx <br>
SRP Run experiment: SRXxxxxxxx <br>
BioProject reference: PRNJAxxxxxx
</blockquote>

#### **3- Retrieve dataset informations**

Go to the <a href='https://www.ncbi.nlm.nih.gov/Traces/study/'>SRA (Sequence Read Archive) Run Selector</a> portal from the NCBI.
> Another option is the <a href='https://trace.ncbi.nlm.nih.gov/Traces/sra/'>SRA</a> website itself but it seems deprecated. The NCBI probably made an update last year (2020) as many other institutes did.

Using the search bar with your SRA accession number, open the page of the dataset you choose. 

Down the page, you can select de samples you want amoung the full list. <br>
Whether you select samples or want to take the entire dataset, download the `Metadata`and the `Accession List` from the `Select` section in the middle of the page.
> Add an accession reference at the beginning of the file names (`SraRunTable.txt` and `SRR_Acc_List.txt` respectively) to distinguish series when you saved the files.

---
## <b>II- Download data files with SRA Toolkit (<i>sra-tools</i>)</b>

Alternatively to use of sra command lines, we could retrieve the list of lines to download fastq files from [sra-explorer web interface](https://ewels.github.io/sra-explorer/) developped by Phil Ewels <a href="https://www.biostars.org/p/366721/#366722"><i>as a fun little side project</i></a>.

### **1- Tools versions**

We load needed tools and display their current versions.

In [2]:
module load sra-tools pigz
# module load sra-tools/2.10.3 pigz/2.3.4

echo "===== sra-tools modules ====="
prefetch --version
vdb-validate --version
fasterq-dump --version
echo "===== compression tool ====="
pigz --version
echo ""

===== sra-tools modules =====

"prefetch" version 2.10.3


vdb-validate : 2.10.3


"fasterq-dump" version 2.10.3

===== compression tool =====
pigz 2.3.4



### **2- Download SRA files with accession list**

<ul class="alert alert-block alert-info">
    <li>
        about sra toolkit <a href="https://www.ncbi.nlm.nih.gov/sra/docs/sradownload/">previous</a> and <a href="https://hpc.nih.gov/apps/sratoolkit.html">latest releases</a>
    </li>
    <li>
    another available option is to use <a href="https://www.biostars.org/p/366721/#366722">sra-explorer web interface</a> to retrieve <code>wget</code> or <code>curl</code> bash command lines
    </li>
</ul>

We will dowload the ``.sra`` files (compressed ones) using the accession list we have downloaded before.

In [3]:
accesslist="${gohome}Data/info/16samples_SRR_Acc_List.txt"

Let's call this file ``accesslist`` for now on.  

To see inside and know which references will be fetched.

In [4]:
cat "${accesslist}"

SRR7430744
SRR7430741
SRR7430706
SRR7430707
SRR7430708
SRR7430709
SRR7430710
SRR7430711
SRR7430712
SRR7430713
SRR7430739
SRR7430740
SRR7430738
SRR7430742
SRR7430743
SRR7430745

We will store downloaded files in ``Data/`` in a subfolder called ``sra``.

In [5]:
mkdir -p "${gohome}Data/sra/"

``prefetch`` is verbose, we can see dates and how it goes directly into the notebook output. As it will take some time, we will eventually disconnect and loose tracks. To prevent information lost, we <a href="https://www.cyberciti.biz/faq/how-to-write-the-output-into-the-file-in-linux/">export</a> and write screen lines in a file (here called <code>logfile</code>).
<blockquote>
    <code>&#38;>></code> to capture both standard output and error output and append a file <br>
    <code>|&#38;</code> to capture both standard output and error output to append a file, still diplaying information on screen <br>
    <code>tee</code> read from standard input and write to standard output and files <br>
    <code>-a</code> for <code>tee</code> command to append file instead of overwriting it
</blockquote>

Then, to launch <code>prefetch</code> tool with this list.

In [6]:
logfile="${gohome}Data/sra-files_retrieval.log"
echo "Screen output is redirected to ${logfile}."

# as time command does not redirect output
echo "operation starting by $(date)" >> ${logfile}

time for srrnum in $(cat "${accesslist}"); do

    echo "=== starting for ${srrnum}..." |& tee -a ${logfile}
    date &>> ${logfile}

    prefetch -O "${gohome}Data/sra/" \
             ${srrnum} \
             &>> ${logfile}

    echo "... done $(date) ===" |& tee -a ${logfile}

done

echo "operation finished by $(date)" >> ${logfile}

Screen output is redirected to /shared/projects/gonseq/Building/Data/sra-files_retrieval.log.
=== starting for SRR7430744...
... done Tue May 11 17:21:01 CEST 2021 ===
=== starting for SRR7430741...
... done Tue May 11 17:24:09 CEST 2021 ===
=== starting for SRR7430706...
... done Tue May 11 17:25:39 CEST 2021 ===
=== starting for SRR7430707...
... done Tue May 11 17:27:06 CEST 2021 ===
=== starting for SRR7430708...
... done Tue May 11 17:28:56 CEST 2021 ===
=== starting for SRR7430709...
... done Tue May 11 17:30:17 CEST 2021 ===
=== starting for SRR7430710...
... done Tue May 11 17:31:53 CEST 2021 ===
=== starting for SRR7430711...
... done Tue May 11 17:33:37 CEST 2021 ===
=== starting for SRR7430712...
... done Tue May 11 17:35:07 CEST 2021 ===
=== starting for SRR7430713...
... done Tue May 11 17:36:38 CEST 2021 ===
=== starting for SRR7430739...
... done Tue May 11 17:38:08 CEST 2021 ===
=== starting for SRR7430740...
... done Tue May 11 17:39:50 CEST 2021 ===
=== starting for S

``prefetch`` options we use or you can add:
<blockquote>
    <code>-O</code> to specify destination folder <br>
    <code>-t</code> to specify where the temporary files are to be written  <br>
    <code>-c</code> to check if presents else download file (default, look into current working directory) <br>
    <code>-f</code> to force overwrite existing files instead of fail <br>

</blockquote>

Let's check that we have all expected files before going on.  
``prefetch`` tool put every ``.sra`` file in a folder named by its accession number, so we will use the ``tree`` command to visualise them all.

In [4]:
tree "${gohome}Data/sra/" >> ${logfile}
tree "${gohome}Data/sra/" | grep "files"

16 directories, 16 files


If we have same numbers between samples references and downloaded file, it's a good beginning point. Let's check there is no issue with those files. 

### **3- Check sra files integrity**

We will use the tool ``vdb-validate`` from ``sra-tools`` to know if files matches those on web archive.

In [9]:
logfile="${gohome}Data/sra-files_integrity.log"
echo "Screen output is redirected to ${logfile}"

echo "operation starting by $(date)" >> ${logfile}

time for srrfile in $(find "${gohome}" -name "*.sra" | sort); do

    echo "... working on $(basename ${srrfile}) file..." |& tee -a ${logfile}
    vdb-validate "${srrfile}" &>> ${logfile}
                 
done

echo "operation finished by $(date)" >> ${logfile}

Screen output is redirected to /shared/projects/gonseq/Building/Data/sra-files_integrity.log
... working on SRR7430706.sra file...
... working on SRR7430707.sra file...
... working on SRR7430708.sra file...
... working on SRR7430709.sra file...
... working on SRR7430710.sra file...
... working on SRR7430711.sra file...
... working on SRR7430712.sra file...
... working on SRR7430713.sra file...
... working on SRR7430738.sra file...
... working on SRR7430739.sra file...
... working on SRR7430740.sra file...
... working on SRR7430741.sra file...
... working on SRR7430742.sra file...
... working on SRR7430743.sra file...
... working on SRR7430744.sra file...
... working on SRR7430745.sra file...

real	1m4.760s
user	0m42.092s
sys	0m16.848s


Now, you can either open ``.log`` file within a text editor (left panel, browse for file and right click on it) or display it in this notebook to read.  
A shorter option consists in showing only lines that concern ``.sra`` files (extension with ``.`` and ``' `` either we will have comment lines):

In [11]:
cat ${logfile} | grep ".sra' "

2021-05-12T09:49:28 vdb-validate.2.10.3 info: Table 'SRR7430706.sra' metadata: md5 ok
2021-05-12T09:49:32 vdb-validate.2.10.3 info: Table 'SRR7430706.sra' is consistent
2021-05-12T09:49:32 vdb-validate.2.10.3 info: Table 'SRR7430707.sra' metadata: md5 ok
2021-05-12T09:49:36 vdb-validate.2.10.3 info: Table 'SRR7430707.sra' is consistent
2021-05-12T09:49:36 vdb-validate.2.10.3 info: Table 'SRR7430708.sra' metadata: md5 ok
2021-05-12T09:49:40 vdb-validate.2.10.3 info: Table 'SRR7430708.sra' is consistent
2021-05-12T09:49:40 vdb-validate.2.10.3 info: Table 'SRR7430709.sra' metadata: md5 ok
2021-05-12T09:49:44 vdb-validate.2.10.3 info: Table 'SRR7430709.sra' is consistent
2021-05-12T09:49:44 vdb-validate.2.10.3 info: Table 'SRR7430710.sra' metadata: md5 ok
2021-05-12T09:49:49 vdb-validate.2.10.3 info: Table 'SRR7430710.sra' is consistent
2021-05-12T09:49:49 vdb-validate.2.10.3 info: Table 'SRR7430711.sra' metadata: md5 ok
2021-05-12T09:49:52 vdb-validate.2.10.3 info: Table 'SRR7430711.sra' 

Another option is to count for lines that contains ***ok*** (each sample with 1 for ``md5`` and 5 for ``checksums``) or ***is consistent*** (1 per ``.sra`` file).

In [13]:
cat ${logfile} | grep "md5 ok" | wc -l
echo $(( $(cat ${logfile} | grep "checksums ok" | wc -l) / 5))
cat ${logfile} | grep "is consistent" | wc -l

16
16
16


### **4- ``.fastq.gz`` files generation: introduction**

<ul class="alert alert-block alert-info">
    <li>
        about <a href="https://github.com/ncbi/sra-tools/wiki/08.-prefetch-and-fasterq-dump">prefetch and fasterq-dump</a>
    </li>
    <li>
        <a href="https://usegalaxy.eu/">usegalaxy.eu</a>: first section (<i>Get Data</i>), <br>workflow called <b>Faster Download and Extract Reads in FASTQ format from NCBI SRA</b>
    </li>
    <li>
        about <a href="https://zlib.net/pigz/pigz.pdf">pigz</a> for multithreading
    </li>
</ul>

The ``.sra`` files downloaded in the previous cells are 'small' files compared to ``.fastq`` files. The idea is to download small amounts of data to avoid network issues during the process... but we can't start our analyses on them. We have to re-create raw ``.fastq`` files first. <br>

To that purpose, ``fasterq-dump`` tool (derived from ``fastq-dump`` in previous sra-tools releases) works on a ``.sra`` file and decompress it in a fastq file (default, 1 per read so 2 for pair end data, PE). <br>

The default command line is: ``fasterq-dump filename.sra``.   
It performs reads separation in 3 files (``--split-3`` option in ``fastq-dump`` tool) for paired-end data: 1 file per read direction for mate reads and another one for remaining unmated reads.  
We can add some options to specify folders to be used, otherwise generated files will be written in current working directory. <br>
<blockquote>
    <code>--outdir</code>, to know where to find the <code>.fastq</code> files <br>
    <code>--temp</code>, to control where temporary files are, indeed temporarily, stored (and check they are removed after :-) ) <br>
</blockquote>

As ``.fastq`` files are quite big files, we'll add the compression step right after any ``.fastq`` files is created. The tool, called ``pigz``, compresses files in the ``.gzip`` format that can be handled downstream. <br>
This tool is based on ``gzip`` tool (so an anagram or an acronym for *Pigz is gzip*?). It allows to have several processes in parallel to work on a the same file... and it goes faster than ``gzip`` tool, even if it still takes quite some time: up to 8-9 minutes for the biggest file among the 16 selected in ``gonseq`` (SRR7430745). <br>
<blockquote>
    <code>-p number</code> or <code>--processes number</code> to allow up to <code>number</code> compression threads (default is the number of online processors, or 8 if unknown)
</blockquote>

### **5- To obtain ``.fastq.gz`` files from ``.sra``**

Using ``--outdir`` in ``fasterq-dump`` command, we will place created files in following folder:

In [14]:
outfolder="${gohome}Data/fastq/raw/"
mkdir -p "${outfolder}"

Be patient while running following *big loop*'s cell!

In [15]:
logfile="${gohome}Data/sra-files_creation_fastqgz.log"
echo "Screen output is redirected to ${logfile}."

# as time command does not redirect output
echo "operation starting by $(date)" >> ${logfile}

time for srrnum in $(ls "${gohome}Data/sra/"); do
    
    echo "========== Processing sample ${srrnum} ===========" |& tee -a ${logfile}
    srrfolder="${gohome}Data/sra/${srrnum}"
    fasterq-dump --outdir ${outfolder} \
                 --temp ${outfolder} \
                 ${srrfolder} \
                 &>> ${logfile}
    
    # don't assume there are 1 or 2 fastq by sample (3rd one if unmated reads)
    for fullfile in $(ls "${outfolder}"*.fastq); do
    
        echo "... compressing $(basename ${fullfile})..." |& tee -a ${logfile}
        date >> ${logfile}
        pigz --processes 9 ${fullfile}
        date >> ${logfile}
    done
    
done

echo "operation finished by $(date)" >> ${logfile}
ls -lh "${outfolder}" >> ${logfile}
echo "Step done"

Screen output is redirected to /shared/projects/gonseq/Building/Data/sra-files_creation_fastqgz.log.
========== Processing sample SRR7430706 ===========
... compressing SRR7430706_1.fastq...
... compressing SRR7430706_2.fastq...
========== Processing sample SRR7430707 ===========
... compressing SRR7430707_1.fastq...
... compressing SRR7430707_2.fastq...
========== Processing sample SRR7430708 ===========
... compressing SRR7430708_1.fastq...
... compressing SRR7430708_2.fastq...
========== Processing sample SRR7430709 ===========
... compressing SRR7430709_1.fastq...
... compressing SRR7430709_2.fastq...
========== Processing sample SRR7430710 ===========
... compressing SRR7430710_1.fastq...
... compressing SRR7430710_2.fastq...
========== Processing sample SRR7430711 ===========
... compressing SRR7430711_1.fastq...
... compressing SRR7430711_2.fastq...
========== Processing sample SRR7430712 ===========
... compressing SRR7430712_1.fastq...
... compressing SRR7430712_2.fastq...
===

## <b>III- Files and folders summary when leaving</b>

Let's count the number of files in our destination folder.

In [17]:
ls "${outfolder}" | wc -l

32


... and check the arborescence of our folder with the Unix command `tree`.  
<blockquote>
    Adding <code>-L</code> option and a number allows to stop digging into the tree folder... and let the output be still readable. <br>
    To list only directories, use option <code>-d</code>
</blockquote>

In [19]:
tree -d -L 3 "${gohome}"

/shared/projects/gonseq/Building/
├── Data
│   ├── fastq
│   │   └── raw
│   ├── info
│   └── sra
│       ├── SRR7430706
│       ├── SRR7430707
│       ├── SRR7430708
│       ├── SRR7430709
│       ├── SRR7430710
│       ├── SRR7430711
│       ├── SRR7430712
│       ├── SRR7430713
│       ├── SRR7430738
│       ├── SRR7430739
│       ├── SRR7430740
│       ├── SRR7430741
│       ├── SRR7430742
│       ├── SRR7430743
│       ├── SRR7430744
│       └── SRR7430745
├── Pipeline
└── Results

23 directories


To know how many disk space files take (either shortest output or detailed one):

In [20]:
du -ch -d1 "${outfolder}"

75G	/shared/projects/gonseq/Building/Data/fastq/raw/
75G	total


or

In [21]:
ls -lh "${outfolder}"

total 75G
-rw-rw----+ 1 bnoblet bnoblet 2.4G May 12 14:35 SRR7430706_1.fastq.gz
-rw-rw----+ 1 bnoblet bnoblet 2.5G May 12 14:35 SRR7430706_2.fastq.gz
-rw-rw----+ 1 bnoblet bnoblet 2.1G May 12 14:43 SRR7430707_1.fastq.gz
-rw-rw----+ 1 bnoblet bnoblet 2.1G May 12 14:43 SRR7430707_2.fastq.gz
-rw-rw----+ 1 bnoblet bnoblet 2.5G May 12 14:50 SRR7430708_1.fastq.gz
-rw-rw----+ 1 bnoblet bnoblet 2.5G May 12 14:50 SRR7430708_2.fastq.gz
-rw-rw----+ 1 bnoblet bnoblet 2.2G May 12 14:58 SRR7430709_1.fastq.gz
-rw-rw----+ 1 bnoblet bnoblet 2.3G May 12 14:58 SRR7430709_2.fastq.gz
-rw-rw----+ 1 bnoblet bnoblet 2.6G May 12 15:05 SRR7430710_1.fastq.gz
-rw-rw----+ 1 bnoblet bnoblet 2.7G May 12 15:05 SRR7430710_2.fastq.gz
-rw-rw----+ 1 bnoblet bnoblet 2.3G May 12 15:14 SRR7430711_1.fastq.gz
-rw-rw----+ 1 bnoblet bnoblet 2.3G May 12 15:14 SRR7430711_2.fastq.gz
-rw-rw----+ 1 bnoblet bnoblet 2.0G May 12 15:21 SRR7430712_1.fastq.gz
-rw-rw----+ 1 bnoblet bnoblet 2.0G May 12 15:21 SRR7430712_2.fastq.gz
-rw-rw----


---
___

Now we can go on to get an eye on data and check samples quality.  
  
**=> Lecture 2 : Raw data quality** 